In [5]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

SECRET_KEY = os.getenv('GOOGLE_PALM_API_KEY')

In [6]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=SECRET_KEY, temperature=0.2)
poem = llm('write a poem on my love for dosa')
print(poem)

Oh, dosa, my beloved,
You are the food of my dreams.
Your crispy exterior and fluffy interior
Are a match made in heaven.

I love the way you smell,
So fresh and inviting.
I love the way you taste,
So light and flavorful.

I love the way you look,
So golden and delicious.
I love the way you make me feel,
So happy and content.

Oh, dosa, my beloved,
You are the best thing that has ever happened to me.
I will love you forever and a day.

Here's to you, dosa,
The food of my heart.


In [9]:
from langchain.sql_database import SQLDatabase

db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [14]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra large size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra large size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XL'
SQLResult: [(10,)]
Answer:10
> Finished chain.


In [15]:
qns1

'10'

In [13]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('404'),)]
Answer:404
> Finished chain.


![Alt text](image.png)

The LLM gave the wrong answer as the table name specified are incorrect thus it strongly represents the real life scenario 

In [16]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('23036'),)]
Answer:23036
> Finished chain.


In [17]:
qns3 = db_chain.run("If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN start_date AND end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN start_date AND end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

The above query was so complex it just failed terribly.

![Alt text](image-1.png)

In [18]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('30237.750000'),)]
Answer:30237.75
> Finished chain.


In [19]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('34011'),)]
Answer:34011
> Finished chain.


In [20]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(95,), (91,)]
Answer:95
> Finished chain.


Once again the above answer was wrong.

![Alt text](image-2.png)

In [21]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('186'),)]
Answer:186
> Finished chain.


# FEW SHOT LEARNING

In [22]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1}, # Structure for training dataset construction
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [23]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

e = embeddings.embed_query("How many white color Levi's shirt I have?")

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 1.19MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<?, ?B/s] 
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 20.7MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 73.1kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 204kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:30<00:00, 2.97MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<?, ?B/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<?, ?B/s] 
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 756kB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<?, ?B/s] 
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<?, ?B/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 619kB/s]
modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 


In [24]:
print(e)

[0.0036103567108511925, 0.07093453407287598, -0.0027510228101164103, 0.0009241139050573111, 0.05407032370567322, -0.00922128465026617, 0.11021784693002701, 0.010788778774440289, 0.011513525620102882, -0.015146060846745968, -0.012995836324989796, -0.06515860557556152, 0.0762801542878151, -0.03886345773935318, 0.005233656615018845, 0.05658937990665436, -0.01418895460665226, -0.0635509118437767, -0.08680646121501923, -0.07036565989255905, 0.037653811275959015, -0.04434247314929962, 0.0198671855032444, 0.009024493396282196, -0.11289289593696594, -0.04166072607040405, 0.030624860897660255, 0.001947940792888403, -0.006058659870177507, -0.059762079268693924, -0.0038959442172199488, 0.004214531276375055, -0.005716186016798019, 0.025438934564590454, -0.03920840844511986, -0.11984361708164215, 0.06245684623718262, -0.014693434350192547, -0.036157380789518356, 0.03521592170000076, 0.027826763689517975, -0.03201691806316376, 0.01522752083837986, 0.04073508456349373, 0.018323203548789024, 0.0656067

Text Blob generation

In [25]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [26]:
to_vectorize[0]

"How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 10"

In [28]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [29]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_Selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)

example_Selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '10',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '34011',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [30]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [31]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [32]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [33]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=['Question', 'SQLQuery', 'SQLResult', 'Answer',],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [34]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_Selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], # These variables are used in prompt template.
)

In [35]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [36]:
new_chain("How much is the price of the inventory for all smal size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all smal size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('23036'),)]
Answer:23036
> Finished chain.


{'query': 'How much is the price of the inventory for all smal size t-shirts?',
 'result': '23036'}

No it is working correctly!!!!

In [37]:
new_chain("How much is the price of all extra small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all extra small size t-shirts?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE size = 'XS'
SQLResult: [(Decimal('13351'),)]
Answer:13351
> Finished chain.


{'query': 'How much is the price of all extra small size t-shirts?',
 'result': '13351'}